## Cargar y parsear archivo

In [1]:
import pandas as pd

In [2]:
# Leer el archivo .txt, cada línea es una secuencia
with open("/content/AFP_data.txt", "r") as f:
    # elimina saltos de línea vacíos
    secuencias = [line.strip() for line in f if line.strip()]

In [3]:
# Crear el dataframe con dos columnas: 'sequence' y 'label'
df = pd.DataFrame({
    "sequence": secuencias,
    "label": [1] * len(secuencias)  # asigna 1 a todas las filas
})

In [4]:
display(df.head())

,sequence,label
0,AAGMGFFGAR,1
1,AAHGACHVRNGKHMCFCYF,1
2,AAKKSFIIKQKLAKAKNQNRPLPQWFRLKTNNTIRYNAKRRHWRRT...,1
3,AALKGCWTKSIPPKPCSGKR,1
4,AALRGCWTKSIPPKPCPGKR,1


## Verificar duplicados

In [6]:
df.shape

(2324, 2)

In [7]:
df["sequence"].unique().shape

(2324,)

In [8]:
df = df.drop_duplicates().copy()

In [9]:
df.shape

(2324, 2)

## Verificar canónicos

In [10]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [13]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["sequence"]]

In [14]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [15]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 2324


In [16]:
display(df_validas)

,sequence,label
0,AAGMGFFGAR,1
1,AAHGACHVRNGKHMCFCYF,1
2,AAKKSFIIKQKLAKAKNQNRPLPQWFRLKTNNTIRYNAKRRHWRRT...,1
3,AALKGCWTKSIPPKPCSGKR,1
4,AALRGCWTKSIPPKPCPGKR,1
...,...,...
2319,YRGGYTGPIPRPPPIGRPPLRLVVCACYRLSVSDARNCCIKFGSCC...,1
2320,YSRCQLQGFNCVVRSYGLPTIPCCRGLTCRSYFPGSTYGRCQRY,1
2321,YSYKKIDCGGACAARCRLSSRPRLCNRACGTCCARCNCVPPGTSGN...,1
2322,YVPKIPKPQPNKPNFPSFPGHGPFNPHASRFPRSPKDNGKIVFDAK...,1


In [17]:
df_validas.to_csv('ETFC_labeled.csv', index=False, header=True)

## Obtener metadatos

In [18]:
import json

In [19]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [20]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [21]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [22]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "ETFC")

In [23]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [24]:
export_json("metadata.json", dict_metadata)